In [1]:
import sys
sys.path.append('..')
from boundariesdetectioncnn.train import train_model
from boundariesdetectioncnn import configs 
import os

In [10]:
from boundariesdetectioncnn.data import dataloaders 

In [34]:
import torch 

def load_weights(input, name, epochs):
    
    if input not in configs.INPUTS.keys():
        raise ValueError('Select a proper input name')
        
    if name == 'mel':
        subdir = name
    elif name == 'sslm':
        subdir = name + '/' + input
    elif name == 'mel_sslm_combined':
        subdir = name + '/' + input + input2
    elif name == 'mel_2sslm_combined':
        subdir = name + '/' + input + input2
    elif name == 'mel_4sslm_combined':
        subdir = name + '/' + input + input2 + input3 + input4
            
    # load model
    model = configs.load_model(name)
    
    # load pretrained weights
    weights_path = os.path.join("../pretrained_weights/" + subdir, "saved_model_" + str(epochs) + "epochs.bin")
    if not os.path.exists(weights_path):
        raise ValueError('Pretrained weights for the selected epochs does not exist.')
    else:
        model.load_state_dict(torch.load(weights_path))
    
    return model

In [35]:
load_weights(input='mel', name='mel', epochs=180)

CNN_Fusion(
  (cnn1): CNN_1(
    (conv1): Conv2d(1, 32, kernel_size=(5, 7), stride=(1, 1), padding=(2, 3))
    (pool1): MaxPool2d(kernel_size=(5, 3), stride=(5, 1), padding=(1, 1), dilation=1, ceil_mode=False)
  )
  (cnn2): CNN_2(
    (conv2): Conv2d(32, 64, kernel_size=(3, 5), stride=(1, 1), padding=(1, 6), dilation=(1, 3))
    (dropout1): Dropout2d(p=0.5, inplace=False)
    (lineal1): Conv1d(1024, 128, kernel_size=(1,), stride=(1,))
    (dropout2): Dropout2d(p=0.5, inplace=False)
    (lineal2): Conv1d(128, 1, kernel_size=(1,), stride=(1,))
  )
)

In [21]:
def predict_song_testdataset(input, name, epochs, song_id, input2='', input3='', input4=''):
    
    if input not in configs.INPUTS.keys():
        raise ValueError('Select a proper input name')
    
    #build test dataloaders
    dataset, data_loader = dataloaders.build_dataloader(batch_size=configs.ParamsConfig.BATCH_SIZE, input=input, run='test')
        
    # load model
    model = configs.load_model(name)
    
    if name == 'mel':
        subdir = name
    elif name == 'sslm':
        subdir = name + '/' + input
    elif name == 'mel_sslm_combined':
        subdir = name + '/' + input + input2
    elif name == 'mel_2sslm_combined':
        subdir = name + '/' + input + input2
    elif name == 'mel_4sslm_combined':
        subdir = name + '/' + input + input2 + input3 + input4
            
    # load model
    model = configs.load_model(name)
    
    # load pretrained weights
    weights_path = os.path.join("../pretrained_weights/" + subdir, "saved_model_" + str(epochs) + "epochs.bin")
    print(weights_path)
    if not os.path.exists(weights_path):
        raise ValueError('Pretrained weights for the selected epochs does not exist.')
    else:
        model.load_state_dict(torch.load(weights_path))
        model.eval()

predict_song_testdataset(input='mel', name='mel', epochs=180, song_id=3, input2=None, input3=None, input4=None)

../pretrained_weights/mel\saved_model_180epochs.bin


NameError: name 'torch' is not defined

In [40]:
import plotly.graph_objs as go
from plotly.offline import plot
import numpy as np
import sklearn
import mir_eval
from scipy import signal

In [43]:
def predict_song_testdataset(input, name, epochs, song_id, input2=None, input3=None, input4=None):
    
    if input not in configs.INPUTS.keys():
        raise ValueError('Select a proper input name')
    
    #build test dataloaders
    dataset, data_loader = dataloaders.build_dataloader(batch_size=configs.ParamsConfig.BATCH_SIZE, input=input, run='test')
        
    # load model
    model = load_weights(input, name, epochs)
    model.eval()
    

    mel = np.expand_dims(mels_dataset[song_id][0], 0)
    mel = torch.Tensor(mel)
    pred = model(torch.Tensor(mls))
    pred = pred.view(-1,1)
    pred = torch.sigmoid(pred)
    pred_new = pred.detach().numpy()
    pred_new = pred_new[:,0]

    #------------------------------------------------------------------------------
    label = dataset[song_id][2]
    label = label[1:]
    reference = np.array((np.copy(label[:-1]), np.copy(label[1:]))).T

    peak_position = signal.find_peaks(pred_new, height=delta, distance=lamda)[0] #array of peaks
    peaks_position = ((peak_position-configs.ParamsConfig.PADDING_FACTOR)*configs.ParamsConfig.POOLING_FACTOR*configs.ParamsConfig.HOP_LENGHT)/configs.ParamsConfig.SAMPLING_RATE
    for i in range(len(peaks_position)):
        if peaks_position[i] < 0:
            peaks_position[i] = 0

    pred_positions = np.array((np.copy(peaks_position[:-1]), np.copy(peaks_position[1:]))).T
    repeated_list = []
    for j in range(pred_positions.shape[0]):
            if pred_positions[j,0] == pred_positions[j,1]:
                repeated_list.append(j)
    pred_positions = np.delete(pred_positions, repeated_list, 0)


    P, R, F, TP = mir_eval.segment.detection(reference, pred_positions, window=configs.ParamsConfig.WINDOW, beta=beta, trim=False)
    print("Threshold", delta)
    print('P =',P,'R =',R,'F =',F)

    TP = len(TP)
    FP = ((1 - P)*TP) / P
    FN = ((1 - R)*TP) / R

    print("True Positives:", TP)
    print("False Positives:", FP)
    print("False Negatives:", FN)

    delta_array = np.zeros_like(sslms_dataset[song_id][1])
    vector = np.arange(sslms_dataset[song_id][0].shape[2])
    #------------------------------------------------------------------------------
    #Plot out vs labels
    for i in range(len(delta_array)):
        delta_array[i] = delta
    plt.plot(vector, delta_array*300, color='aqua')
    plt.plot(vector, dataset[song_id][1]*300, 'r-', label='Labels')
    plt.plot(vector, pred[:,0].detach().numpy()*300, 'w-', label='Output')
    plt.imshow(dataset[song_id][0][0,...], origin = 'lower', aspect=1)
    plt.ylabel("lag bins")
    matplotlib.rcParams.update({'font.size': 10})

    plt.show()


    #------------------------------------------------------------------------------
    #Plot out vs labels
    for i in range(len(delta_array)):
        delta_array[i] = delta
    
    trace1 = go.Scatter(x = vector,
                        y = sslms_dataset[song_id][1],
                        mode = 'lines',
                        name = 'labels',
                        marker = dict(color = 'rgba(72, 141, 244, 1)') #blue
                        )
    trace2 = go.Scatter(x = vector,
                        y = pred_new,
                        mode = 'lines',
                        name = 'predictions',
                        marker = dict(color = 'rgba(15, 194, 129, 1)') #green
                        )
    trace3 = go.Scatter(x = vector,
                        y = delta_array,
                        mode = 'lines',
                        name = 'delta',
                        marker = dict(color = 'rgba(229, 183, 31, 1)') #yellow
                        )
    trace4 = go.Scatter(x = peak_position,
                        y = [pred_new[j] for j in peak_position],
                        mode = 'markers',
                        name = 'estimated peaks',
                        marker = dict(color = 'rgba(240, 87, 57, 1)') #red
                        )
    data = [trace1, trace2, trace3, trace4]
    layout = dict(title = 'Song' + song_id + 'SALAMI 2.0  ' + epochs + ' epochs',
                  xaxis= dict(title= 'Time (seconds)',ticklen= 5,zeroline= False))
    fig = dict(data = data, layout = layout)
    plot(fig)

In [44]:
predict_song_testdataset(input='mel', name='mel', epochs=180, song_id=3, input2=None, input3=None, input4=None)

NameError: name 'mels_dataset' is not defined